In [ ]:
from pathlib import Path
from rosbags.typesys import get_typestore, Stores, get_types_from_msg
from rosbags.rosbag2 import Reader
import pandas as pd

typestore = get_typestore(Stores.ROS2_HUMBLE)

def register_msg_file(typestore, msg_path: str, full_type_name: str):
    msg_text = Path(msg_path).read_text()
    add_types = get_types_from_msg(msg_text, full_type_name)
    typestore.register(add_types)

# --- Register RTK msg ---
register_msg_file(
    typestore,
    "src/rtk_driver_msgs/msg/CustomRTK.msg",
    "rtk_driver_msgs/msg/CustomRTK"
)

# --- Register GPS msg (from the other package) ---
register_msg_file(
    typestore,
    "src/gps_driver_interfaces/msg/gps_msg.msg",
    "gps_driver_interfaces/msg/gps_msg"
)


def bag_to_df(bag_path, typestore):
    rows = []

    with Reader(bag_path) as reader:
        for connection, timestamp, rawdata in reader.messages():

            if connection.topic != "/gps":
                continue

            msg = typestore.deserialize_cdr(rawdata, connection.msgtype)

            rows.append({
                "bag_time_ns": int(timestamp),
                "stamp_sec": int(msg.header.stamp.sec),
                "stamp_nanosec": int(msg.header.stamp.nanosec),
                "frame_id": msg.header.frame_id,

                "latitude": float(msg.latitude),
                "longitude": float(msg.longitude),
                "altitude": float(msg.altitude),
                "hdop": float(msg.hdop),

                "utm_easting": float(msg.utm_easting),
                "utm_northing": float(msg.utm_northing),

                "utc": str(msg.utc),
                "zone": str(msg.zone),
                "letter": str(msg.letter),
            })

    return pd.DataFrame(rows)

FileNotFoundError: [Errno 2] No such file or directory: 'gps_driver_msgs/msg/GpsMsg.msg'